Dans cet notebook, nous abordons les statistiques descriptives sur les incidents de sécurité sur le réseau de la SCNF

In [22]:
import requests
import pandas as pd
url_api="https://ressources.data.sncf.com/api/v2/catalog/datasets/incidents-securite/exports/json?limit=-1&offset=0&lang=fr&timezone=Europe%2FParis"
incident= pd.read_json(url_api)
incident.head()

,id,origine,numero_isic,type_event,date,nature,region,lieu,niveau_gravite
0,18212,Mobilités,0,Dépassement de la vitesse limite de plus de 40...,2015-01-12,Dépassement de vitesse supérieur à 40 km/h,RA,Montbrison (42),NaN
1,18152,Réseau,None,RFC,2015-01-14,"Un TGV franchi le PN 25 barrières hautes, la s...",None,Lunel (34),NaN
2,18196,Réseau,None,Défaillance voie,2015-01-19,Défaut de géométrie nécessitant l'arrêt des ci...,LR,Entre Sète et Le Castellas,NaN
3,18203,Mobilités,17347,Franchissement intempestif d'un signal d'arrêt...,2015-01-20,Le signal C348 est franchit fermé par le train...,CEN,Orléans,NaN
4,18379,Réseau,17477,Engagement intempestif d'un train sur une voie...,2015-02-04,Engagement intempestif d'une circulation sur u...,RA,Lyon Perrache (69),NaN
